# WEB SCRAPING

<p align = "justify">
    This project is about web scraping using this two python library: Selenium and Openyxl.
    Follow these steps bello.
</p>

### STEP 1: requirements

In [43]:
!pip3 install selenium openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [44]:
# Simula o uso do navegador
from selenium import webdriver

# Permite encontrar informações dentro de sites
from selenium.webdriver.common.by import By

# Permite criar planilhas
import openpyxl

### STEP 2: Development

In [52]:
# Abrindo o navegador e o site
    # Inicializando o navegador pelo webdriver.
    # Estou utilizando o navegador Chrome (por isso webdriver.Chrome())
    # Entrarei na seguinte loja: 

driver = webdriver.Chrome()
driver.get('https://www.kabum.com.br/computadores/monitores/monitor-gamer')

# Coletando os dados
    # xpath é uma forma de encontrar elementos dentro de um site
    # xpath strucutre = //tag[@attribute='value']
    # Title xpath = //span[@class='sc-d79c9c3f-0 nlmfp sc-93fa31de-16 bBOYrL nameCard']
    # Price xpath = //span[@class='sc-6889e656-2 bYcXfg priceCard']

titles = driver.find_elements(By.XPATH,"//span[@class='sc-d79c9c3f-0 nlmfp sc-93fa31de-16 bBOYrL nameCard']")
promotional_prices = driver.find_elements(By.XPATH,"//span[@class='sc-6889e656-2 bYcXfg priceCard']")



# Criando a planilha
planilha = openpyxl.Workbook()

# Criando a página 'produtos'
planilha.create_sheet('produtos')

# Selecionando a página 'produtos' e inserindo os rótulos (cabeçalhos)
sheet_produtos = planilha['produtos']
sheet_produtos['A1'].value = 'PRODUTO'
sheet_produtos['B1'].value = 'PREÇO PROMOCIONAL'

# Inserindo os dados na planilha (somente tuplas de nome, preço com valors diferentes de null para manter consistência)
for title, price in zip(titles, promotional_prices):
    sheet_produtos.append([title.text, price.text])

planilha.save('database.xlsx')

